## Fine tuining modal with dp_prompt and dp-paraphraser

In [ ]:
# class code for LLM-based Differential Privacy mechanisms

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList, pipeline
from transformers import BartTokenizer, BartModel, BartForConditionalGeneration
from transformers.models.bart.modeling_bart import BartDecoder
import torch
import torch.nn.functional as nnf
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from pathlib import Path

import mpmath
from mpmath import mp

import nltk
nltk.download('punkt', quiet=True)

class ClipLogitsProcessor(LogitsProcessor):
  def __init__(self, min=-100, max=100):
    self.min = min
    self.max = max

  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
    scores = torch.clamp(scores, min=self.min, max=self.max)
    return scores
  
class DPPrompt():
    model_checkpoint = None
    min_logit = None
    max_logit = None
    sensitivity = None
    logits_processor = None

    tokenizer = None
    model = None
    device = None

    def __init__(self, model_checkpoint="~/google/flan-t5-base", min_logit=-19.22705113016047, max_logit=7.48324937989716):
        self.model_checkpoint = model_checkpoint

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_checkpoint).to(self.device)

        self.min_logit = min_logit
        self.max_logit = max_logit
        self.sensitivity = abs(self.max_logit - self.min_logit)
        self.logits_processor = LogitsProcessorList([ClipLogitsProcessor(self.min_logit, self.max_logit)])


    def prompt_template_fn(self, doc):
        prompt = "Document : {}\nParaphrase of the document :".format(doc)
        return prompt
    
    def privatize(self, text, epsilon=100):
        temperature = 2 * self.sensitivity / epsilon

        prompt = self.prompt_template_fn(text)

        model_inputs = self.tokenizer(prompt, max_length=512, truncation=True, return_tensors="pt").to(self.device)
        output = self.model.generate(
            **model_inputs,
            do_sample = True,
            top_k=0,
            top_p=1.0,
            temperature=temperature,
            max_new_tokens=len(model_inputs["input_ids"][0]),
            logits_processor=self.logits_processor)

        private_text = self.tokenizer.decode(output[0], skip_special_tokens = True )
        return private_text
    
class DPParaphrase():
    model_checkpoint = None
    min_logit = None
    max_logit = None
    sensitivity = None
    logits_processor = None

    tokenizer = None
    model = None
    device = None

    def __init__(self, model_checkpoint=("models/gpt2-paraphraser"), min_logit=-96.85249956065758, max_logit=-8.747697966442914):
        self.model_checkpoint = model_checkpoint

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_checkpoint)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_checkpoint, pad_token_id=self.tokenizer.eos_token_id).to(self.device)

        self.min_logit = min_logit
        self.max_logit = max_logit
        self.sensitivity = abs(self.max_logit - self.min_logit)
        self.logits_processor = LogitsProcessorList([ClipLogitsProcessor(self.min_logit, self.max_logit)])

        self.pipe = pipeline(task="text-generation", model=self.model, 
                             tokenizer=self.tokenizer, 
                             logits_processor=self.logits_processor, 
                             device=self.device,
                             pad_token_id=self.tokenizer.eos_token_id)

    def privatize(self, text, epsilon=100):
        temperature = 2 * self.sensitivity / epsilon

        prompt = text+ " >>>>> "
        length = len(self.tokenizer(prompt)["input_ids"])
    
        private_text = self.pipe(prompt, max_new_tokens=length, temperature=temperature)[0]["generated_text"]
        return private_text.replace(prompt, "").replace(prompt.strip(), "").replace(u'\xa0', u' ').replace(">", "").strip()
   
class DPBart():
    model = None
    decoder = None
    tokenizer = None

    sigma = None
    num_sigmas = None
    c_min = None
    c_max = None
    delta = None

    def __init__(self, model='facebook/bart-base', num_sigmas=1/2):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.tokenizer = BartTokenizer.from_pretrained(model)
        self.model = BartModel.from_pretrained(model).to(self.device)
        self.decoder = BartForConditionalGeneration.from_pretrained(model).to(self.device)

        self.delta = 1e-5
        self.sigma = 0.2
        self.num_sigmas = num_sigmas
        self.c_min = -self.sigma
        self.c_max = self.num_sigmas * self.sigma

    def clip(self, vector):
        return torch.clip(vector, self.c_min, self.c_max)

    # https://github.com/trusthlt/dp-bart-private-rewriting/
    def calibrateAnalyticGaussianMechanism_precision(self, epsilon, delta, GS, tol = 1.e-12):
        """
        High-precision version of `calibrateAnalyticGaussianMechanism`.
        Should not be used for epsilon values above 5000.
        """

        if epsilon <= 1000:
            mp.dps = 500  # works for epsilon = 1000
        elif epsilon <= 2500 and epsilon > 1000:
            mp.dps = 1100  # works for epsilon = 2500
        else:
            mp.dps = 2200  # works for epsilon = 5000

        def Phi(t):
            return 0.5*(1.0 + mpmath.erf(t/mpmath.sqrt(2.0)))

        def caseA(epsilon,s):
            return Phi(mpmath.sqrt(epsilon*s)) - mpmath.exp(epsilon)*Phi(-mpmath.sqrt(epsilon*(s+2.0)))

        def caseB(epsilon,s):
            return Phi(-mpmath.sqrt(epsilon*s)) - mpmath.exp(epsilon)*Phi(-mpmath.sqrt(epsilon*(s+2.0)))

        def doubling_trick(predicate_stop, s_inf, s_sup):
            while(not predicate_stop(s_sup)):
                s_inf = s_sup
                s_sup = 2.0*s_inf
            return s_inf, s_sup

        def binary_search(predicate_stop, predicate_left, s_inf, s_sup):
            s_mid = s_inf + (s_sup-s_inf)/2.0
            while(not predicate_stop(s_mid)):
                if (predicate_left(s_mid)):
                    s_sup = s_mid
                else:
                    s_inf = s_mid
                s_mid = s_inf + (s_sup-s_inf)/2.0
            return s_mid

        delta_thr = caseA(epsilon, 0.0)

        if (delta == delta_thr):
            alpha = 1.0

        else:
            if (delta > delta_thr):
                predicate_stop_DT = lambda s : caseA(epsilon, s) >= delta
                function_s_to_delta = lambda s : caseA(epsilon, s)
                predicate_left_BS = lambda s : function_s_to_delta(s) > delta
                function_s_to_alpha = lambda s : mpmath.sqrt(1.0 + s/2.0) - mpmath.sqrt(s/2.0)

            else:
                predicate_stop_DT = lambda s : caseB(epsilon, s) <= delta
                function_s_to_delta = lambda s : caseB(epsilon, s)
                predicate_left_BS = lambda s : function_s_to_delta(s) < delta
                function_s_to_alpha = lambda s : mpmath.sqrt(1.0 + s/2.0) + mpmath.sqrt(s/2.0)

            predicate_stop_BS = lambda s : abs(function_s_to_delta(s) - delta) <= tol

            s_inf, s_sup = doubling_trick(predicate_stop_DT, 0.0, 1.0)
            s_final = binary_search(predicate_stop_BS, predicate_left_BS, s_inf, s_sup)
            alpha = function_s_to_alpha(s_final)

        sigma = alpha*GS/mpmath.sqrt(2.0*epsilon)

        return float(sigma)
    
    def noise(self, vector, epsilon, delta=1e-5, method="gaussian"):
        k = vector.shape[-1]
        if method == "laplace":
            sensitivity = 2 * self.sigma * self.num_sigmas * k
            Z = torch.from_numpy(np.random.laplace(0, sensitivity / epsilon, size=k))
        elif method == 'gaussian':
            sensitivity = 2 * self.sigma * self.num_sigmas * np.sqrt(k)
            scale = np.sqrt((sensitivity**2 / epsilon**2) * 2 * np.log(1.25 / self.delta))
            Z = torch.from_numpy(np.random.normal(0, scale, size=k))
        elif method == "analytic_gaussian":
            sensitivity = 2 * self.sigma * self.num_sigmas * np.sqrt(k)
            analytic_scale = self.calibrateAnalyticGaussianMechanism_precision(epsilon, self.delta, sensitivity)
            Z = torch.from_numpy(np.random.normal(0, analytic_scale, size=k))
        return vector + Z

    def privatize(self, text, epsilon=100, method="gaussian"):
        inputs = self.tokenizer(text, max_length=512, truncation=True, return_tensors="pt").to(self.device)
        num_tokens = len(inputs["input_ids"][0])

        enc_output = self.model.encoder(**inputs)
        enc_output["last_hidden_state"] = self.noise(self.clip(enc_output["last_hidden_state"].cpu()), epsilon=epsilon, delta=self.delta, method=method).float().to(self.device)

        dec_out = self.decoder.generate(encoder_outputs=enc_output, max_new_tokens=num_tokens)
        private_text = self.tokenizer.decode(dec_out[0], skip_special_tokens=True)
        return private_text.strip()


In [ ]:
# Optional installs (uncomment if needed)
# !pip install -q transformers datasets evaluate accelerate

import os
import evaluate
from datasets import load_dataset, DatasetDict, Value
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

# -----------------------------
# 0) REQUIRED: your privatizers
# -----------------------------
# M, N, K must exist and expose:
#   M.dpmlm_rewrite(text: str, epsilon: int) -> str
#   N.Privatize(text: str, epsilon: int) -> str
#   K.privatize(text: str, epsilon: int) -> str
#
# from your_module import M, N, K
# M, N, K = ...
# For safety, add NLTK resources if your methods rely on them:
try:
    import nltk
    for res in ["punkt", "punkt_tab"]:
        try: nltk.data.find(f"tokenizers/{res}")
        except LookupError: nltk.download(res)
except Exception:
    pass

# -----------------------------
# 1) Config
# -----------------------------
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LEN = 128
BATCH = 16
LR = 5e-5
EPOCHS = 1
WD = 0.01
OUTPUT_BASE = "dv3-priv"
EPSILONS = [10, 25, 50, 100, 250]   # change as you like
TECHNIQUES = ["dpmlm", "privN", "privK"]  # M, N, K

# For IMDB-Genres local file
IMDB_PATH = "data/imdb_genres.csv"  # change to your file path
IMDB_DELIM = ","                    # "," for CSV or "\t" for TSV
IMDB_TEXT_COL = "text"
IMDB_LABEL_COL = "label"

# Map datasets to their input text columns
# (name, subset, text1, text2, notes)
DATASETS = [
    # SNLI (3-class): premise/hypothesis, filter label == -1
    ("snli", None, "premise", "hypothesis"),
    # QQP (binary)
    ("glue", "qqp", "question1", "question2"),
    # PAWS labeled_final (binary)
    ("paws", "labeled_final", "sentence1", "sentence2"),
    # IMDB-Genres (multi-class) — local file, single text column
    ("imdb_genres_local", None, IMDB_TEXT_COL, None),
]

# -----------------------------
# 2) Utilities
# -----------------------------
def make_training_args(**base_kwargs):
    """Shim: transformers v5 uses eval_strategy, v4 uses evaluation_strategy."""
    try:
        return TrainingArguments(eval_strategy="epoch", **base_kwargs)
    except TypeError:
        return TrainingArguments(evaluation_strategy="epoch", **base_kwargs)

def _to_str(x):
    if x is None: return ""
    return x if isinstance(x, str) else str(x)

def privatize_fn_builder(tech: str, eps: int, col1: str, col2: str | None):
    """Return a batched map function that rewrites text columns."""
    def fn(batch):
        out = {}
        # primary column
        if tech == "dpmlm":
            out[col1] = [ _to_str(M.dpmlm_rewrite(_to_str(t), eps)) for t in batch[col1] ]
        elif tech == "privN":
            out[col1] = [ _to_str(N.Privatize(_to_str(t), eps))      for t in batch[col1] ]
        elif tech == "privK":
            out[col1] = [ _to_str(K.privatize(_to_str(t), eps))      for t in batch[col1] ]
        else:
            raise ValueError(f"Unknown technique: {tech}")

        # secondary column (if any)
        if col2 is not None:
            if tech == "dpmlm":
                out[col2] = [ _to_str(M.dpmlm_rewrite(_to_str(t), eps)) for t in batch[col2] ]
            elif tech == "privN":
                out[col2] = [ _to_str(N.Privatize(_to_str(t), eps))      for t in batch[col2] ]
            elif tech == "privK":
                out[col2] = [ _to_str(K.privatize(_to_str(t), eps))      for t in batch[col2] ]

        # keep label untouched if present
        if "label" in batch:
            out["label"] = batch["label"]
        return out
    return fn

def tokenize_builder(tok, col1, col2, is_regression=False):
    """Tokenize and attach labels -> 'labels' field."""
    def fn(batch):
        if col2 is None:
            enc = tok(batch[col1], truncation=True, max_length=MAX_LEN)
        else:
            enc = tok(batch[col1], batch[col2], truncation=True, max_length=MAX_LEN)
        if "label" in batch:
            enc["labels"] = [float(x) for x in batch["label"]] if is_regression else batch["label"]
        return enc
    return fn

def compute_accuracy_builder(num_labels: int):
    """Classification accuracy metric."""
    acc = evaluate.load("accuracy")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = logits.argmax(axis=-1)
        return acc.compute(predictions=preds, references=labels)
    return compute_metrics

# -----------------------------
# 3) Dataset loaders (standardize to train/validation splits)
# -----------------------------
def load_snli():
    ds = load_dataset("snli")
    # Filter invalid labels (-1)
    ds = ds.filter(lambda ex: ex["label"] != -1)
    # SNLI has 'train', 'validation', 'test'
    return DatasetDict({
        "train": ds["train"],
        "validation": ds["validation"]
    })

def load_qqp():
    ds = load_dataset("glue", "qqp")
    return DatasetDict({
        "train": ds["train"],
        "validation": ds["validation"]
    })

def load_paws_final():
    ds = load_dataset("paws", "labeled_final")
    return DatasetDict({
        "train": ds["train"],
        "validation": ds["validation"]
    })

def load_imdb_genres_local(path: str, delim: str):
    if not os.path.exists(path):
        raise FileNotFoundError(f"IMDB-Genres file not found at: {path}")
    ds = load_dataset("csv", data_files={"train": path, "validation": path}, delimiter=delim)
    # If you actually have a separate validation file, change data_files accordingly.
    return ds

def get_loader(name, subset):
    if name == "snli": return load_snli()
    if name == "glue" and subset == "qqp": return load_qqp()
    if name == "paws" and subset == "labeled_final": return load_paws_final()
    if name == "imdb_genres_local": return load_imdb_genres_local(IMDB_PATH, IMDB_DELIM)
    raise ValueError(f"Unsupported dataset: {name}/{subset}")

# -----------------------------
# 4) Main: loop datasets × techniques × epsilons
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
data_collator = DataCollatorWithPadding(tokenizer)

for (ds_name, ds_subset, s1, s2) in DATASETS:
    print(f"\n================ Dataset: {ds_name}{'/' + ds_subset if ds_subset else ''} =================\n")
    ds = get_loader(ds_name, ds_subset)

    # Arrow schema safety — force text columns to string
    if s1 in ds["train"].column_names:
        ds = ds.cast_column(s1, Value("string"))
    if s2 and s2 in ds["train"].column_names:
        ds = ds.cast_column(s2, Value("string"))

    # Determine number of labels (classification). Assume 'label' exists and is categorical/int.
    # For local IMDB, ensure labels are ints (0..C-1).
    if "label" not in ds["train"].column_names:
        raise ValueError(f"'label' column missing in {ds_name}. Please add/rename accordingly.")
    label_feat = ds["train"].features["label"]
    if hasattr(label_feat, "names") and label_feat.names:
        num_labels = len(label_feat.names)
    else:
        # Fallback: infer from data if no names (e.g., custom IMDB)
        # You can replace with your known num_classes
        unique_labels = set(ds["train"][IMDB_LABEL_COL] if ds_name=="imdb_genres_local" else ds["train"]["label"])
        num_labels = len(unique_labels)

    # Build model fresh for each run (dataset/tech/epsilon)
    for tech in TECHNIQUES:
        for eps in EPSILONS:
            tag = f"{ds_name}{('-' + ds_subset) if ds_subset else ''}-{tech}-eps{eps}"
            print(f"\n---- {tag} ----")

            # 1) Privatize
            priv_fn = privatize_fn_builder(tech, eps, s1, s2)
            ds_priv = DatasetDict({
                "train": ds["train"].map(priv_fn, batched=True),
                "validation": ds["validation"].map(priv_fn, batched=True),
            })

            # 2) Tokenize and attach labels
            tok_fn = tokenize_builder(tokenizer, s1, s2, is_regression=False)
            remove_cols = [c for c in ds_priv["train"].column_names if c not in {s1, s2, "label"}]
            enc = DatasetDict({
                "train": ds_priv["train"].map(tok_fn, batched=True, remove_columns=remove_cols),
                "validation": ds_priv["validation"].map(tok_fn, batched=True, remove_columns=remove_cols),
            })

            # 3) Model + Trainer
            model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

            args = make_training_args(
                output_dir=f"{OUTPUT_BASE}-{tag}",
                save_strategy="epoch",
                learning_rate=LR,
                per_device_train_batch_size=BATCH,
                per_device_eval_batch_size=BATCH,
                num_train_epochs=EPOCHS,
                weight_decay=WD,
                logging_dir=f"{OUTPUT_BASE}-{tag}/logs",
                report_to="none",
                load_best_model_at_end=False,
                logging_steps=50,
                # fp16=True,  # uncomment if your GPU supports it
            )

            trainer = Trainer(
                model=model,
                args=args,
                train_dataset=enc["train"],
                eval_dataset=enc["validation"],
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_accuracy_builder(num_labels),
            )

            # 4) Train + Evaluate
            trainer.train()
            metrics = trainer.evaluate()
            print(f"RESULT [{tag}] -> {metrics}")
